In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import EarlyStopping

'''
#Example script to generate text from Nietzsche's writings.
At least 20 epochs are required before the generated text
starts sounding coherent.
It is recommended to run this script on GPU, as recurrent
networks are quite computationally intensive.
If you try this script on new data, make sure your corpus
has at least ~100k characters. ~1M is better.
'''
'''
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
'''

In [7]:
'''
path = get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))
'''

text = open("input.txt", 'r', encoding='utf-8').read()

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))
print('Text Length:', len(text))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1
    
print(x.shape)
print(y.shape)
print(x[0])

total chars: 65
nb sequences: 371785
Text Length: 1115394
Vectorization...
(371785, 40, 65)
(371785, 65)
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


In [6]:
# build the model: a single LSTM
print('Build model...')
'''
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
'''

'''def get_model(batch_size):
    model = keras.Sequential()
    model.add(keras.layers.LSTM(1024, 
                                input_shape = (maxlen, len(chars)), 
                                batch_size = batch_size, 
                                stateful=True, 
                                return_sequences=True))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.LSTM(1024, stateful=True))
    model.add(keras.layers.Dropout(0.2))
    model.add(keras.layers.Dense(len(chars), activation='softmax'))
    return model
model = get_model(128)'''

model = keras.Sequential()
model.add(keras.layers.LSTM(1024, 
                                input_shape = (maxlen, len(chars)),
                                return_sequences=True))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.LSTM(1024))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

print(model.summary())

Build model...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 1024)          4464640   
_________________________________________________________________
dropout (Dropout)            (None, 40, 1024)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024)              8392704   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 65)                66625     
Total params: 12,923,969
Trainable params: 12,923,969
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs=None): #changed from (epoch, _)
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    #for diversity in [0.2, 0.5, 1.0, 1.2]:
    for diversity in [1.0]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [37]:
class MyCustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None): #changed from (epoch, _)
        # Function invoked at end of each epoch. Prints generated text.
        print()
        print('----- Generating text after Epoch: %d' % epoch)

        start_index = random.randint(0, len(text) - maxlen - 1)
        #for diversity in [0.2, 0.5, 1.0, 1.2]:
        for diversity in [1.0]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + sentence + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()

In [38]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

'''tf.keras.callbacks.LambdaCallback(
    on_epoch_begin=None, on_epoch_end=None, on_batch_begin=None, on_batch_end=None,
    on_train_begin=None, on_train_end=None, **kwargs
)
'''

model.fit(x, y,
          batch_size=128,
          epochs=15,
          callbacks=[MyCustomCallback()])

Train on 371785 samples
Epoch 1/15
  6912/371785 [..............................] - ETA: 4:15:05 - loss: 3.5063
----- Generating text after Epoch: 0
----- diversity: 1.0
----- Generating with seed: "All places yield to him ere he sits down"
All places yield to him ere he sits downlacle i tlnt EogdiAvea
 rnvLlnholhm rpr wheoysgyna
SsaUiOerH, eesnt  titoataohsofteo nntlsooo  ,tmsn.b,unraae mh
noOnbn
seHe
ochoo;vl Wld r hu,OecinH snr,ttamgnt
p:towtgla adH oreoanerofido tHomyit sS
Aiymeot rlyo
gas
o- hno soml eanis loe tu n ab&;ns tlaoi s wLot?nluaeev r Httg u otu oosoUli omnnsstistiseetlott
Er:,eeeoeia,K 
c
tnd oa  d,ywmas.is  Iylet  oanisukmnuooKtlxunn
mey barae ohl
eiPagtio
  6912/371785 [..............................] - ETA: 7:37:11 - loss: 3.5063

KeyboardInterrupt: 